<a href="https://colab.research.google.com/github/Ninevv/RAG_Project/blob/Zineb/Langchain_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pypdf python-dotenv
!pip install transformers
!pip install torch
!pip install -q datasets loralib sentencepiece
!pip install -q einops accelerate langchain bitsandbytes
!pip install sentence_transformers
!pip install llama-index
!%pip install --upgrade --quiet  langchain langchain-openai faiss-cpu tiktoken

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4

In [ ]:
import os

In [ ]:
corpus_of_documents = [
    "Take a leisurely walk in the park and enjoy the fresh air.",
    "Visit a local museum and discover something new.",
    "Attend a live music concert and feel the rhythm.",
    "Go for a hike and admire the natural scenery.",
    "Have a picnic with friends and share some laughs.",
    "Explore a new cuisine by dining at an ethnic restaurant.",
    "Take a yoga class and stretch your body and mind.",
    "Join a local sports league and enjoy some friendly competition.",
    "Attend a workshop or lecture on a topic you're interested in.",
    "Visit an amusement park and ride the roller coasters."
]

In [ ]:
def jaccard_similarity(query, document):
    query = query.lower().split(" ")
    document = document.lower().split(" ")
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [ ]:
def return_response(query, corpus):
    similarities = []
    for doc in corpus:
        similarity = jaccard_similarity(user_input, doc)
        similarities.append(similarity)
    return corpus_of_documents[similarities.index(max(similarities))]

In [ ]:
user_prompt = "What is a leisure activity that you like?"

In [ ]:
user_input = "I like to hike"

In [ ]:
return_response(user_input, corpus_of_documents)

In [ ]:
!pip install huggingface_hub

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "HF_TOKEN"

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint

In [ ]:
import torch

In [ ]:
# If you want more control, you will need to define the tokenizer and model.
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer

In [ ]:
import bitsandbytes
import accelerate

In [ ]:
# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", quantization_config=quantization_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

# With pipeline, just specify the task and the model id from the Hub.
from transformers import pipeline
pipe = pipeline("text-generation", model=model,tokenizer=tokenizer, max_new_tokens=10)

In [ ]:
pipe("What is the name of this restaurant?")

#Indexing : Load#

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [ ]:
len(docs[0].page_content)

In [ ]:
print(docs[0].page_content[:500])

#Indexing : Split#

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [ ]:
len(all_splits)

In [ ]:
len(all_splits[0].page_content)

In [ ]:
all_splits[10].metadata

#Context aware splitter rag q&a quickstart#

#Indexing : Store#

In [ ]:
from langchain_community.vectorstores import Chroma #vs FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# create the open-source embedding function
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding_function, persist_directory="./chroma_db")

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")
len(retrieved_docs)

In [ ]:
print(retrieved_docs[0].page_content)

In [ ]:
# query it
query = "What did the president say about Ketanji Brown Jackson"
docs = vectorstore.similarity_search(query)

# print results
print(docs[0].page_content)

#Retrieval and Generation#

In [ ]:
llm = model

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages

In [ ]:
print(example_messages[0].content)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough



def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.stream

In [ ]:
for chunk in rag_chain.stream("What is Task Decomposition"):
    print(chunk, end="", flush=True)

#Optional : costumize prompt#

In [ ]:
#costumize prompt
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

In [ ]:
# save to disk
db2 = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_db")
docs = db2.similarity_search(query)

# load from disk
db3 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)
docs = db3.similarity_search(query)
print(docs[0].page_content)